In [1]:
import sys
sys.path.append('..')
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer, seed_everything
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score

from torch_explain.models.explainer import MuExplainer
from torch_explain.logic import test_explanation, complexity
from experiments.data.load_datasets import load_mimic

In [2]:
x, y, concept_names = load_mimic()

dataset = TensorDataset(x, y)

train_size = int(len(dataset)*0.5)
val_size = (len(dataset)-train_size) // 2
test_size = len(dataset)-train_size-val_size
train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])
train_loader = DataLoader(train_data, batch_size=train_size)
val_loader = DataLoader(val_data, batch_size=val_size)
test_loader = DataLoader(test_data, batch_size=test_size)

n_concepts = next(iter(train_loader))[0].shape[1]
n_classes = 2

print(len(train_loader))
print(len(val_loader))
print(len(test_loader))
print(concept_names)
print(n_concepts)
print(n_classes)

1
1
1
['aline_flg', 'gender_num', 'sepsis_flg', 'chf_flg', 'afib_flg', 'renal_flg', 'liver_flg', 'copd_flg', 'cad_flg', 'stroke_flg', 'mal_flg', 'resp_flg', 'abg_count_LOW', 'abg_count_NORMAL', 'abg_count_HIGH', 'age_LOW', 'age_NORMAL', 'age_HIGH', 'bmi_LOW', 'bmi_NORMAL', 'bmi_HIGH', 'bun_first_LOW', 'bun_first_NORMAL', 'bun_first_HIGH', 'chloride_first_LOW', 'chloride_first_NORMAL', 'chloride_first_HIGH', 'creatinine_first_LOW', 'creatinine_first_NORMAL', 'creatinine_first_HIGH', 'day_icu_intime_num_LOW', 'day_icu_intime_num_NORMAL', 'day_icu_intime_num_HIGH', 'hgb_first_LOW', 'hgb_first_NORMAL', 'hgb_first_HIGH', 'hour_icu_intime_LOW', 'hour_icu_intime_NORMAL', 'hour_icu_intime_HIGH', 'hr_1st_LOW', 'hr_1st_NORMAL', 'hr_1st_HIGH', 'icu_los_day_LOW', 'icu_los_day_NORMAL', 'icu_los_day_HIGH', 'iv_day_1_LOW', 'iv_day_1_NORMAL', 'iv_day_1_HIGH', 'map_1st_LOW', 'map_1st_NORMAL', 'map_1st_HIGH', 'pco2_first_LOW', 'pco2_first_NORMAL', 'pco2_first_HIGH', 'platelet_first_LOW', 'platelet_first

In [3]:
seed_everything(42)

base_dir = f'./results/mimic-ii/explainer'
os.makedirs(base_dir, exist_ok=True)

n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
results_list = []
for split, (trainval_index, test_index) in enumerate(skf.split(x.cpu().detach().numpy(), y.cpu().detach().numpy())):
    print(f'Split [{split+1}/{n_splits}]')
    x_trainval, x_test = torch.FloatTensor(x[trainval_index]), torch.FloatTensor(x[test_index])
    y_trainval, y_test = torch.LongTensor(y[trainval_index]), torch.LongTensor(y[test_index])
    x_train, x_val, y_train, y_val = train_test_split(x_trainval, y_trainval, test_size=0.2, random_state=42)
    print(f'{len(y_train)}/{len(y_val)}/{len(y_test)}')
    
    train_data = TensorDataset(x_train, y_train)
    val_data = TensorDataset(x_val, y_val)
    test_data = TensorDataset(x_test, y_test)
    train_loader = DataLoader(train_data, batch_size=train_size)
    val_loader = DataLoader(val_data, batch_size=val_size)
    test_loader = DataLoader(test_data, batch_size=test_size)
    
    checkpoint_callback = ModelCheckpoint(dirpath=base_dir, monitor='val_loss', save_top_k=1)
    trainer = Trainer(min_epochs=100, max_epochs=100, gpus=1, auto_lr_find=True, deterministic=True,
                      check_val_every_n_epoch=1, default_root_dir=base_dir,
                      weights_save_path=base_dir, callbacks=[checkpoint_callback])
    model = MuExplainer(n_concepts=n_concepts, n_classes=n_classes, concept_activation='identity_bool', 
                        l1=0.001, prune_epoch=50, fan_in=10, lr=0.01)
    
    trainer.fit(model, train_loader, val_loader)
    model.freeze()
    model_results = trainer.test(model, test_dataloaders=test_loader)
    results = model.explain_class(train_loader, val_loader, target_class=1, 
                                  topk_explanations=3, concept_names=concept_names)
    results['model_accuracy'] = model_results[0]['test_acc']
    
    results_list.append(results)


Global seed set to 42
c:\users\pietr\anaconda3\envs\deep-logic\lib\site-packages\pytorch_lightning\utilities\distributed.py:52: UserWarning: Checkpoint directory ./results/mimic-ii/explainer exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Split [1/10]
1278/320/178



  | Name  | Type       | Params
-------------------------------------
0 | loss  | NLLLoss    | 0     
1 | model | Sequential | 1.0 K 
-------------------------------------
1.0 K     Trainable params
0         Non-trainable params
1.0 K     Total params
0.004     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.8587570786476135, 'test_acc_epoch': 0.8587570786476135}
--------------------------------------------------------------------------------


In [4]:
results_df = pd.DataFrame(results_list)
results_df

,target_class,explanation,explanation_accuracy,explanation_fidelity,explanation_complexity,model_accuracy
0,1,stroke_flg | (afib_flg & resp_flg & age_HIGH &...,0.843750,0.843750,21,0.882023
1,1,stroke_flg | (afib_flg & stroke_flg & age_HIGH...,0.834375,0.834375,21,0.887640
2,1,(afib_flg & stroke_flg & age_HIGH & ~renal_flg...,0.865625,0.865625,30,0.882023
3,1,(stroke_flg & ~weight_first_NORMAL) | (stroke_...,0.856250,0.856250,22,0.865169
4,1,stroke_flg | (afib_flg & stroke_flg & age_HIGH...,0.856250,0.856250,21,0.865169
5,1,sapsi_first_HIGH | (stroke_flg & resp_flg & ag...,0.853125,0.853125,21,0.865169
6,1,stroke_flg | (stroke_flg & age_HIGH & sofa_fir...,0.856250,0.856250,21,0.824859
7,1,stroke_flg | (afib_flg & stroke_flg & age_HIGH...,0.837500,0.837500,21,0.875706
8,1,(stroke_flg & age_HIGH) | (stroke_flg & age_HI...,0.850000,0.850000,22,0.875706
9,1,stroke_flg | (stroke_flg & age_HIGH & hour_icu...,0.843750,0.843750,21,0.858757


In [5]:
results_df.mean()

target_class               1.000000
explanation_accuracy       0.849687
explanation_fidelity       0.849687
explanation_complexity    22.100000
model_accuracy             0.868222
dtype: float64